In [103]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [104]:
data = 'https://www.kaggle.com/blastchar/telco-customer-churn'


In [105]:
df = pd.read_csv(r"C:\Users\TALIGENT\Downloads\data-week-3.csv.csv")
df.head().T # Transponemos para ver todas lsa columnas



,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [106]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

features_categoricas = list(df.dtypes[df.dtypes == 'object'].index)

for c in features_categoricas:
    df[c] = df[c].str.lower().str.replace(' ', '_')

df.totalcharges = pd.to_numeric(df.totalcharges, errors= 'coerce') #ignora los errores y lo convierte en numerico
df.totalcharges = df.totalcharges.fillna(0)
df.totalcharges.isnull().sum()

df.churn  = (df.churn == 'yes').astype(int) # verdadero pasa 1 y otro 0

In [107]:
# Configuracion del Marco de Trabajo
# Usaremos Scikit-Learn para divivir en train - val- test

from sklearn.model_selection import train_test_split

df_full_train, df_test =train_test_split(df, test_size=0.2, random_state=1)#random_state es seed

df_train, df_val =train_test_split(df_full_train, test_size=0.25, random_state=1) # tomamos el 0.25 xq es del 80% del df

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test= df_test.reset_index(drop=True)

y_train = df_train.churn.values
y_val = df_val.churn.values
y_test = df_test.churn.values

del df_train['churn']
del df_val['churn']
del df_test['churn']

#X_train = df_train

In [108]:
# EDA

df_full_train = df_full_train.reset_index(drop = 'True')
df_full_train.isnull().sum()
df_full_train.churn.value_counts(normalize=(True)) #% de cada categoria
# Nos da igual que antes la tasa de abandono (clientes abandonan del total) (para 1 y 0)
global_churn_rate  = round(df_full_train.churn.mean(),2) 


In [109]:
numerical = ['tenure', 'monthlycharges', 'totalcharges' ]
categoricas = [ 'gender', 'seniorcitizen', 'partner', 'dependents',
        'phoneservice', 'multiplelines', 'internetservice',
       'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport',
       'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling',
       'paymentmethod']


df_full_train[categoricas].nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

In [110]:
# Tasa de abandono (churn rate)
# Lo hacemos grupo de valor en variable categorica y la comparamos contra el total 
# y si hay mucha dif es features importante
churn_female = df_full_train[df_full_train.gender == 'female'].churn.mean()
churn_male = df_full_train[df_full_train.gender == 'male'].churn.mean()
churn_partner =df_full_train[df_full_train.partner == 'yes'].churn.mean()
churn_no_partner = df_full_train[df_full_train.partner == 'no'].churn.mean()

In [111]:
# Features importante (tasa de riesgo)
# Division entre tasa grupo / tasa global
# > 1 ===> mayor probabilidad que abandone
# < 1 ===> menor probabilidad que abandone
# Esto nos da una idea en base al promedio general, si es 1 no es que no tiene riesgo
#  sino que es muy parecido al promedio general, al igual si es alto o bajo
churn_female / global_churn_rate 
churn_partner / global_churn_rate 
churn_no_partner / global_churn_rate 


np.float64(1.22151482997322)

In [ ]:
# se hace para poder ver en todas variables categoricas el risk y la diferencia
from IPython.display import display

for c in categoricas:
    print(c)
    df_group = df_full_train.groupby(c).churn.agg(['mean', 'count'])
    df_group ['diff'] = df_group['mean'] - global_churn_rate
    df_group ['risk'] = df_group['mean'] / global_churn_rate
    display(df_group)
    print()
    

In [113]:
# INFORMACION MUTUA
# Cuanto saber de una variable nos reduce la incertidumbre sobre otra   
# Con sklearn nos da un valor por cada feature
from sklearn.metrics import mutual_info_score

mutual_info_score(df_full_train.churn, df_full_train.contract)
mutual_info_score(df_full_train.churn, df_full_train.gender)
mutual_info_score(df_full_train.churn, df_full_train.partner)
# Definimos una funcion que vamos a usar como variable series
def mutual_info_churn_score(series):
    return mutual_info_score (df_full_train.churn, series)

#apply() aplica la funcion que vaya adentro a todas las columnas
mi = df_full_train[categoricas].apply(mutual_info_churn_score)
mi.sort_values(ascending= False) # Ordenamos de mayor a menor y los mas importantes son los primeros



contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
seniorcitizen       0.009410
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64

In [114]:
# CORRELACION (-1 < r > 1)
# Medir la dependecia de una variable de otra
# Correlacion r > 0 aumenta la variable dependiente aumenta la independeiente
# r < 0 aumenta la variable dependiente disminuye la indipendiente
# 0 - 0.2 ==> correlacion baja
# 0,2 - 0.5 ==> correlacion moderada
# 0,5 - 1 ==> correlacion fuerte
df_full_train[numerical].corrwith(df_full_train.churn).abs()


tenure            0.351885
monthlycharges    0.196805
totalcharges      0.196353
dtype: float64

In [115]:
# OBTENERMOS LA MATRIZ X

In [ ]:
# CODIFICACION VARIABLES CATEGORICAS ONE- HOT ENCODING
# Antes manualmente ahora sklearn
from sklearn.feature_extraction import DictVectorizer
train_dict = df_train[categoricas + numerical].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categoricas + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict) 
#dv.get_feature_names_out() # para ver el orden de las features en la matriz

# EXPLICACION
# DictVectorizer :Transforma esos diccionarios en una matriz de números:
# Las variables categóricas se convierten en columnas con 0/1 (one-hot encoding).
# Las variables numéricas se dejan igual.
# fit_transform :
# fit = aprende qué columnas crear (ej: gender=male, gender=female, age).
# transform = genera la matriz con esos valores.

#Lo aplicamos al df validacion
val_dicts = df_val[categoricas + numerical].to_dict(orient= 'records')
X_val = dv.transform(val_dicts) # no aplicamos fit nuevamente


In [117]:
# REGRESION LOGISTICA (explicacion)
#Funcion sigmoid para convertir los valores "y" variable dep de RL en el rango de 0 - 1
# Nos da la probabilidad positiva: que si renuncie, que sea spam , etc
# Es igual a la RL pero al final se aplica la funcion Sigmoide a los resultados
def sigmoide(z):
    return 1 / (1+ np.exp(-z)) # zi = yi = w0 + (xij * wj)  definimos la funcion sigmoide a aplicar a y

# Definimos la funcion regresion logistica
def logistic_regression(xi): # esto es para una observacion en particular
    score = w0

    for j in range(len(w)):
        score = score + xi[j]* w[j]
    
    result = sigmoide(score) # aplicamos funcion sigmoide sobre score = y

    return result # nos devuele la probailidad que sea 1 o positivo


In [118]:
# ENTRAMIENTO REGRESION LOGISTICA CON SKLEARN (Aplicacion)

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs')
model.fit(X_train, y_train)


c:\Users\TALIGENT\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [119]:
# w0 (sesgo)
model.intercept_[0].round(3)

np.float64(-0.109)

In [120]:
# w (los pesos de cada feature)
model.coef_[0].round(3) 

array([ 0.473, -0.175, -0.406, -0.029, -0.078,  0.063, -0.089, -0.081,
       -0.034, -0.073, -0.335,  0.316, -0.089,  0.004, -0.258,  0.141,
        0.009,  0.063, -0.089, -0.081,  0.265, -0.089, -0.284, -0.231,
        0.124, -0.166,  0.058, -0.087, -0.032,  0.071, -0.059,  0.141,
       -0.248,  0.215, -0.12 , -0.089,  0.101, -0.071, -0.089,  0.052,
        0.213, -0.089, -0.232, -0.07 ,  0.   ])

In [193]:
# Predecimos las probalidades [entre 0 - 1] y nos quedamos con la proba de churn  (postiiva abandono)
y_pred_proba = model.predict_proba(X_val)[:, 1]

# Aca obtenemos y_pred que son las decisiones positivas y negativas (probalidades corte 0.5)
y_pred = model.predict(X_val)
# Obtenemos la media de los valores que coinciden
efectividad = (y_val == y_pred).mean()
efectividad
# Efectividad de 80,20 %

np.float64(0.8026969481902059)

In [146]:
# Analisis de los resultados anteriores. Armamos un DF con probalidades, pred, real y aciertos.
df_pred_proba = pd.DataFrame()
df_pred_proba['probabilidad'] = y_pred_proba.round(3)
df_pred_proba['prediccion']  = y_pred
df_pred_proba['real'] = y_val
df_pred_proba['correcto'] = (y_val == y_pred)
df_pred_proba

,probabilidad,prediccion,real,correcto
0,0.009,0,0,True
1,0.205,0,0,True
2,0.212,0,0,True
3,0.544,1,1,True
4,0.214,0,0,True
...,...,...,...,...
1404,0.313,0,0,True
1405,0.039,0,1,False
1406,0.137,0,0,True
1407,0.800,1,1,True


In [ ]:
dict(zip(dv.get_feature_names_out(),model.coef_[0].round(3))) #Matriz con los pesos de c feature
# Anteriormente lo hicimos por sklearn pero si fuera manual se haria:
# sigmoide (w0 + wj * xij) 
# Lo anterior nos da la probalidad que abandone o que pase el hecho
# se aplica la funcoin sigmoide para llevarla al rango [1 - 0]

[{'contract': 'two_year', 'tenure': 72, 'monthlycharges': 115.5},
 {'contract': 'month-to-month', 'tenure': 10, 'monthlycharges': 95.25},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 75.55},
 {'contract': 'month-to-month', 'tenure': 5, 'monthlycharges': 80.85},
 {'contract': 'two_year', 'tenure': 18, 'monthlycharges': 20.1},
 {'contract': 'month-to-month', 'tenure': 4, 'monthlycharges': 30.5},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 75.1},
 {'contract': 'month-to-month', 'tenure': 1, 'monthlycharges': 70.3},
 {'contract': 'two_year', 'tenure': 72, 'monthlycharges': 19.75},
 {'contract': 'month-to-month', 'tenure': 6, 'monthlycharges': 109.9}]

In [ ]:
# USO DEL MODELO

In [ ]:
#Diccionarios
dict_train_full = df_full_train[categoricas + numerical].to_dict(orient='records')
dict_test = df_test[categoricas + numerical].to_dict(orient='records')

#Obtenemos la X
dv_full = DictVectorizer(sparse= False)
dv_full.fit(dict_train_full)
X_train_full = dv_full.transform(dict_train_full)
X_test = dv_full.transform(dict_test)

#Obtenemos la y
y_train_full = df_full_train.churn.values

model_full = LogisticRegression(solver='lbfgs')
model_full.fit(X_train_full, y_train_full)




c:\Users\TALIGENT\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


1409

In [ ]:
y_full_pred = model_full.predict(X_test)

efectividad_full = (y_test == y_full_pred).mean()
# Efec 80,70 %

np.float64(0.8076650106458482)

In [ ]:
#Probamos el modelo para un cliente en particular (el ultimo)
customer = dict_test[-1]
X_customer = dv.transform(customer)

y_cust_prob_pred = model_full.predict_proba(X_customer)[:,1] # probabilidad
y_cust_pred  = model_full.predict(X_customer)

In [223]:
y_test[-1] == y_cust_pred

array([ True])